# Study: Querying facts with GPT-3

This study aims to investigate the utilization of GPT-3 for querying the extracted facts from a database. The specific objectives of this investigation include:

  - Implementing auxiliary mechanisms to store the extracted facts, utilizing a simple Pandas Dataframe for this purpose.

  - Developing prompts that facilitate querying the facts from the database. Remarkably, we will explore how GPT-3 can be leveraged to enhance this querying process.

### Setup

In [1]:
# install openai library
!pip install openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.3 MB/s eta 0:00:00


In [2]:
# import libraries
import getpass
import openai
import pandas as pd

In [3]:
# get and store openai API key
openai.api_key = getpass.getpass('Enter OpenAI API key: ')

Enter OpenAI API key: ··········


### Helper Functions

In [4]:
def gpt3_complete(
    prompt, 
    engine='text-davinci-003', 
    temperature=0.1, 
    max_tokens=200,
    top_p=1.0, 
    frequency_penalty=0, 
    presence_penalty=0, 
    stop=None,
    echo=False
):
    """
    Generates a completion based on the given prompt using the OpenAI GPT-3 model.

    Parameters:
        prompt (str): The text prompt to generate a completion for.
        engine (str): The engine to use for completion. 
        temperature (float): The temperature parameter controlling the randomness of the generated text. 
        max_tokens (int): The maximum number of tokens in the generated completion.
        top_p (float): The cumulative probability threshold for the top-p sampling algorithm. It controls the diversity of the generated text. 
        frequency_penalty (float): The frequency penalty to apply during completion. 
        presence_penalty (float): The presence penalty to apply during completion. 
        stop (str or list): The stop sequence(s) to use for stopping the generation.
        echo (bool): Whether to include the prompt in the generated completion. Default is False.

    Returns:
        completion (str): The generated completion based on the prompt.
    """
    response = openai.Completion.create(
        engine=engine,
        prompt=prompt,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        stop=stop,
        echo=echo
    )
    
    completion = response['choices'][0]['text']
    return completion

In [5]:
def apply_to_examples(
    examples, 
    prompt_func, 
    temperature=0.1,
    frequency_penalty=0,
    presence_penalty=0
):
    """
    Applies the GPT-3 completion to a list of input examples.

    Parameters:
        examples (list): A list of input examples to generate completions for.
        prompt_func (function): A function that takes an example as input and returns a prompt for GPT-3.
        temperature (float): The temperature parameter controlling the randomness of the generated text.
        frequency_penalty (float): The frequency penalty to apply during completion. 
        presence_penalty (float): The presence penalty to apply during completion. 

    Returns:
        results (list): A list of generated completions for each input example.
    """
    results = []
    for example in examples:
        print(f">>> INPUT: {example}")
        result = gpt3_complete(prompt_func(example), temperature=temperature, frequency_penalty=frequency_penalty, presence_penalty=presence_penalty)
        print(f">>> OUTPUT:\n{result}")
        results.append(result)
        print("========================================\n\n")
    return results

In [6]:
def extract_lines_from_result(result):
    """
    Extracts the lines from the result string.

    Parameters:
        result (str): The result string to extract lines from.

    Returns:
        lines (list): A list of extracted lines from the result.
    """
    lines = [line.strip(' -*') for line in result.split('\n') if len(line) > 0]
    return lines

In [7]:
def string_to_tuples(s):
    """"
    Converts a string that looks like a tuple to an actual Python tuple.

    Parameters:
        s (str): The string representation of the tuple.

    Returns:
        tuple_list (list): A list of tuples converted from the string.
    """
    return [eval(s.strip()) for s in extract_lines_from_result(s)]

In [8]:
def extract_terms_from_all_results(results):
    """
    Extracts the terms from the result string.

    Parameters:
        results (list): A list of result strings.

    Returns:
        terms (list): A list of lists, where each inner list contains the extracted terms from each result string.
    """
    terms = []
    for result in results:
        terms.append(extract_lines_from_result(result))
    return terms

### Example Data

Some standard data to exercise our solution below. That shall include information input to the system, as well as related *queries*.

In [9]:
example_information_to_save = ["Flu shot cost = $80", 
                               "things my boss likes: cricket, science and vegetarian food",
                               "my wife wants a vegetarian food book",
                               "sales guy email = jp@example.com",
                               "vanessa's email vanessa@outlook.com, rember to send the ppts she asked",
                               "+55 11 27670-0987 -> pedro whatsapp",
                               "need to buy milk, eggs and bread", 
                               "need to sell old video game, chair",
                               "december receipts for gym: yoga, ballet, ??",
                               "book with KWG the hardware setup",
                               "dog day with the foreign visitors",
                               "event support: we failed :-(",
                               "floor layout: we forgot various details!!",
                               "first aid kit in the reception",
                               "ask the pediatrician about when to start brushing teeth"]

In [10]:
example_queries = ["shopping list", 
                   "what does my boss like?", 
                   "emails",
                   "questions for the pediatrician"]

In [11]:
# following prompt obtained from fact extraction study
def extraction_prompt(x, categories=["Family", "Work", "Friends", "Shopping", 
                                       "Health", "Finance", "Travel", "Home", 
                                       "Pets", "Hobbies", "Other"]):

    prompt =\
f"""
You are tasked with extracting pieces of personal information from various inputs, such as phone numbers, email addresses, names, trivia, reminders, etc. Your goal is to extract and categorize these pieces of information into structured tuples.
Extract the information as tuples with the following format: (Category, Type, People, Key, Value). 
Assume everything mentioned refers to the same thing. 
The extracted information should be categorized according to the allowed categories and types specified below.

Constraints:
  - Allowed Categories: {', '.join(categories)}
  - Allowed Types: "List", "Email", "Phone", "Address", "Document", "Pendency", "Price", "Reminder", "Note", "Doubt", "Wish", "Other"
  - People contain the name or description of the people or organizations concerned, or is empty if no person or organization is mentioned.
  
Example input: "Mom's phone number is 555-555-5555"
Example output: ("Family", "Phone", "mom", "mom's number", "555-555-5555")

Example input: "email of the building administration = adm@example.com"
Example output: ("Work", "Email", "building administration", "email", "adm@example.com")

Example input: "Need to do: lab work, ultrasound, buy aspirin"
Example output: 
("Health", "List", "Self", "to do", "lab work")
("Health", "List", "Self", "to do", "ultrasound")
("Shopping", "List", "Self", "aspirin", "buy")	

Example input: event support: we failed :-(
Example output: 
("Other", "Note", "event support", "failed", "we failed :-(")

Example input: first aid kit in the reception
Example output:
("Home", "Note", "", "first aid kit", "reception")

Example input: december receipts for gym: yoga, ballet, ??
Example output:
("Finance", "Document", "gym", "december receipts", "yoga")
("Finance", "Document", "gym", "december receipts", "ballet")
("Finance", "Document", "gym", "december receipts", "??")

Input: {x}
Output: 
"""
    return prompt 

### Populating Database

The Pandas dataframe will serve as the primary data structure akin to a database. 

In [12]:
input_tuples = [] # list to save the tuples
for input in example_information_to_save:
    for tuples in string_to_tuples(gpt3_complete(extraction_prompt(input))):
        input_tuples.append(tuples)
        
input_tuples

[('Health', 'Price', '', 'flu shot', '$80'),
 ('Work', 'List', 'boss', 'likes', 'cricket'),
 ('Work', 'List', 'boss', 'likes', 'science'),
 ('Work', 'List', 'boss', 'likes', 'vegetarian food'),
 ('Shopping', 'Wish', 'wife', 'vegetarian food book', 'my wife wants'),
 ('Work', 'Email', 'sales guy', 'email', 'jp@example.com'),
 ('Work', 'Email', 'vanessa', "vanessa's email", 'vanessa@outlook.com'),
 ('Work', 'Reminder', '', 'send ppts', 'rember to send the ppts she asked'),
 ('Friends', 'Phone', 'Pedro', 'WhatsApp', '+55 11 27670-0987'),
 ('Shopping', 'List', 'Self', 'milk', 'buy'),
 ('Shopping', 'List', 'Self', 'eggs', 'buy'),
 ('Shopping', 'List', 'Self', 'bread', 'buy'),
 ('Shopping', 'List', 'Self', 'sell', 'old video game'),
 ('Shopping', 'List', 'Self', 'sell', 'chair'),
 ('Finance', 'Document', 'gym', 'december receipts', 'yoga'),
 ('Finance', 'Document', 'gym', 'december receipts', 'ballet'),
 ('Finance', 'Document', 'gym', 'december receipts', '??'),
 ('Other', 'Document', 'KWG',

In [13]:
database = pd.DataFrame(input_tuples, columns=["Category", "Type", "People", "Key", "Value"])
database

,Category,Type,People,Key,Value
0,Health,Price,,flu shot,$80
1,Work,List,boss,likes,cricket
2,Work,List,boss,likes,science
3,Work,List,boss,likes,vegetarian food
4,Shopping,Wish,wife,vegetarian food book,my wife wants
5,Work,Email,sales guy,email,jp@example.com
6,Work,Email,vanessa,vanessa's email,vanessa@outlook.com
7,Work,Reminder,,send ppts,rember to send the ppts she asked
8,Friends,Phone,Pedro,WhatsApp,+55 11 27670-0987
9,Shopping,List,Self,milk,buy


### Prompt Engineering: Querying Information

#### Prompt 1

The current approach proposed is a simplistic solution that requires examining each row of the Pandas dataframe. While it may achieve the desired outcome, it appears to be computationally expensive and inefficient.

In [18]:
def querying_prompt_1(query, example):

    prompt = \
f"""
Determine whether the query "{query}" is related to the tuple "{example}".
Answer (yes/no):
"""

    return prompt

In [20]:
for query in example_queries:
    print(f"INPUT QUERY: {query}\n")
    for i, row in database.iterrows():
        row_string = f"{tuple(row.values)}"
        print(f"INPUT ROW: {row_string}")
        print(f"OUTPUT: {gpt3_complete(querying_prompt_1(query, row_string))}")
    
    print(f"====================================================================")

INPUT QUERY: shopping list

INPUT ROW: ('Health', 'Price', '', 'flu shot', '$80')
OUTPUT: No
INPUT ROW: ('Work', 'List', 'boss', 'likes', 'cricket')
OUTPUT: No
INPUT ROW: ('Work', 'List', 'boss', 'likes', 'science')
OUTPUT: No
INPUT ROW: ('Work', 'List', 'boss', 'likes', 'vegetarian food')
OUTPUT: No
INPUT ROW: ('Shopping', 'Wish', 'wife', 'vegetarian food book', 'my wife wants')
OUTPUT: Yes
INPUT ROW: ('Work', 'Email', 'sales guy', 'email', 'jp@example.com')
OUTPUT: No
INPUT ROW: ('Work', 'Email', 'vanessa', "vanessa's email", 'vanessa@outlook.com')
OUTPUT: No
INPUT ROW: ('Work', 'Reminder', '', 'send ppts', 'rember to send the ppts she asked')
OUTPUT: No
INPUT ROW: ('Friends', 'Phone', 'Pedro', 'WhatsApp', '+55 11 27670-0987')
OUTPUT: No
INPUT ROW: ('Shopping', 'List', 'Self', 'milk', 'buy')
OUTPUT: Yes
INPUT ROW: ('Shopping', 'List', 'Self', 'eggs', 'buy')
OUTPUT: Yes
INPUT ROW: ('Shopping', 'List', 'Self', 'bread', 'buy')
OUTPUT: Yes
INPUT ROW: ('Shopping', 'List', 'Self', 'sell', 

#### Prompt 2


An alternative approach would involve extracting key terms from the query and utilizing them for searching within the dataframe. By identifying and extracting essential terms or keywords from the query, we can narrow down the search scope and potentially improve the efficiency of the search process. This strategy aims to enhance the relevance and accuracy of the search results by focusing on the specific terms that are most relevant to the query.

In [21]:
def querying_prompt_2(query):

    prompt = \
f"""
Extract the main entities (one per line, without bullets) in the following sentence: "{query}"
"""

    return prompt

In [22]:
results = apply_to_examples(example_queries, querying_prompt_2, temperature=0.1)

>>> INPUT: shopping list
>>> OUTPUT:

Shopping 
List


>>> INPUT: what does my boss like?
>>> OUTPUT:

Boss


>>> INPUT: emails
>>> OUTPUT:

Email


>>> INPUT: questions for the pediatrician
>>> OUTPUT:

Pediatrician 
Questions




In [23]:
examples_terms = extract_terms_from_all_results(results)
examples_terms

[['Shopping', 'List'], ['Boss'], ['Email'], ['Pediatrician', 'Questions']]

#### Prompt 3

The current observations indicate that the entity extraction process is functioning effectively. However, there is a consideration regarding the need for synonym expansion. To address this, we propose performing data augmentation as a means to complement the previous prompt.

This approach aims to broaden the search scope and improve the comprehensiveness of the results by considering synonymous terms and expanding the coverage of relevant information.

In [24]:
def querying_prompt_3(term):

    prompt = \
f"""
List some synonyms to the following term: "{term}"
Synonyms (one synonym per line):
"""

    return prompt

In [25]:
augmentation_raw_results = []
for terms in examples_terms:
    augmentation_raw_result = apply_to_examples(terms, querying_prompt_3)
    augmentation_raw_results.append(augmentation_raw_result)

augmentation_raw_results

>>> INPUT: Shopping
>>> OUTPUT:

Purchasing
Acquiring
Buying
Procuring
Obtaining


>>> INPUT: List
>>> OUTPUT:

Inventory
Register
Catalogue
Index
Roll
Table


>>> INPUT: Boss
>>> OUTPUT:

Supervisor
Manager
Chief
Leader
Head
Director
Foreman
Governor
Principal


>>> INPUT: Email
>>> OUTPUT:

Message
Correspondence
Communication
Post
Dispatch


>>> INPUT: Pediatrician
>>> OUTPUT:

Child Doctor
Paediatrician
Infant Specialist
Kids Physician
Youngster Practitioner


>>> INPUT: Questions
>>> OUTPUT:

Inquiries
Queries
Interrogations
Enquiries
Examinations
Pose
Quizzes




[['\nPurchasing\nAcquiring\nBuying\nProcuring\nObtaining',
  '\nInventory\nRegister\nCatalogue\nIndex\nRoll\nTable'],
 ['\nSupervisor\nManager\nChief\nLeader\nHead\nDirector\nForeman\nGovernor\nPrincipal'],
 ['\nMessage\nCorrespondence\nCommunication\nPost\nDispatch'],
 ['\nChild Doctor\nPaediatrician\nInfant Specialist\nKids Physician\nYoungster Practitioner',
  '\nInquiries\nQueries\nInterrogations\nEnquiries\nExaminations\nPose\nQuizzes']]

In [26]:
example_augmentations = []
for raw_result in augmentation_raw_results:
    example_augmentation = extract_terms_from_all_results(raw_result)
    flat_example_augmentation = [item for sublist in example_augmentation for item in sublist]
    example_augmentations.append(flat_example_augmentation)

example_augmentations    

[['Purchasing',
  'Acquiring',
  'Buying',
  'Procuring',
  'Obtaining',
  'Inventory',
  'Register',
  'Catalogue',
  'Index',
  'Roll',
  'Table'],
 ['Supervisor',
  'Manager',
  'Chief',
  'Leader',
  'Head',
  'Director',
  'Foreman',
  'Governor',
  'Principal'],
 ['Message', 'Correspondence', 'Communication', 'Post', 'Dispatch'],
 ['Child Doctor',
  'Paediatrician',
  'Infant Specialist',
  'Kids Physician',
  'Youngster Practitioner',
  'Inquiries',
  'Queries',
  'Interrogations',
  'Enquiries',
  'Examinations',
  'Pose',
  'Quizzes']]

### Querying Demo

In [27]:
def search_dataframe(df, original_terms, augmented_terms):
    """
    Searches the database for the specified terms.
    
    Parameters:
        df (pandas.DataFrame): The dataframe to search.
        original_terms (list): The original search terms.
        augmented_terms (list): The additional search terms.

    Returns:
        df_results (pandas.DataFrame): The dataframe containing the search results.
    """
    all_terms = original_terms + augmented_terms
    df = df.fillna("")

    df_results = None
    for column in df.columns:
        df_result = df[df[column].str.contains("|".join(all_terms), case=False).fillna(False)]
        if df_results is None:
            df_results = df_result
        else:
            df_results = pd.concat([df_results, df_result])
            
    return df_results

In [28]:
database # database created initially

,Category,Type,People,Key,Value
0,Health,Price,,flu shot,$80
1,Work,List,boss,likes,cricket
2,Work,List,boss,likes,science
3,Work,List,boss,likes,vegetarian food
4,Shopping,Wish,wife,vegetarian food book,my wife wants
5,Work,Email,sales guy,email,jp@example.com
6,Work,Email,vanessa,vanessa's email,vanessa@outlook.com
7,Work,Reminder,,send ppts,rember to send the ppts she asked
8,Friends,Phone,Pedro,WhatsApp,+55 11 27670-0987
9,Shopping,List,Self,milk,buy


In [29]:
for i, original_terms in enumerate(examples_terms):
    augmented_terms = example_augmentations[i]
    print(f"Search terms: {original_terms}")
    print(f"Augmented terms: {augmented_terms}")
    print(search_dataframe(database, original_terms, augmented_terms))
    print(f"====================")

Search terms: ['Shopping', 'List']
Augmented terms: ['Purchasing', 'Acquiring', 'Buying', 'Procuring', 'Obtaining', 'Inventory', 'Register', 'Catalogue', 'Index', 'Roll', 'Table']
    Category  Type People                   Key            Value
4   Shopping  Wish   wife  vegetarian food book    my wife wants
9   Shopping  List   Self                  milk              buy
10  Shopping  List   Self                  eggs              buy
11  Shopping  List   Self                 bread              buy
12  Shopping  List   Self                  sell   old video game
13  Shopping  List   Self                  sell            chair
1       Work  List   boss                 likes          cricket
2       Work  List   boss                 likes          science
3       Work  List   boss                 likes  vegetarian food
9   Shopping  List   Self                  milk              buy
10  Shopping  List   Self                  eggs              buy
11  Shopping  List   Self               